https://huggingface.co/akahana/indonesia-sentiment-roberta

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Load Model Using Pipeline

In [12]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", device='cuda', model="akahana/indonesia-sentiment-roberta")

In [28]:
label = pipe([
    "Hi, namaku yasir",
    "Aku akan menaklukan dunia",
    "Dan memerintah orang dengan tangan besi"
])
print(label[0]['label'])
print(label[0]['score'])

NETRAL
0.8895004391670227


# Load Model Directly

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("akahana/indonesia-sentiment-roberta")
model = AutoModelForSequenceClassification.from_pretrained("akahana/indonesia-sentiment-roberta")

In [ ]:
tokenizer([
    "Hi, namaku yasir",
    "Aku akan menaklukan dunia",
    "Dan memerintah orang dengan tangan besi"
])

# Labeling

In [16]:
from pandas import read_csv
from tqdm import tqdm
tqdm.pandas()

df = read_csv('./pubg_preprocessing.csv')
df.rename(columns={"text_lemmatize": "text"}, inplace=True)

In [76]:
def sentimentLabeling(text):
    labeling = pipe(text)
    label = labeling[0]['label']
    score = labeling[0]['score']
    return label, score

In [96]:
labeling = df['text_slangwords'].progress_apply(sentimentLabeling)

100%|██████████| 20000/20000 [05:23<00:00, 61.82it/s]


In [118]:
labeling_zip = list(zip(*labeling))
df['score'] = labeling_zip[1]
df['sentiment'] = labeling_zip[0]

In [119]:
df.head()

,content,text_clean,text_slangwords,text_stopwords,text,score,sentiment
0,"Tencent, saya bukan player pro,. Tapi kami sec...",tencent saya bukan player pro tapi kami secara...,tencent saya bukan player profesional tapi kam...,tencent player profesional mengeluh update bug...,tencent player profesional keluh update bug fr...,0.981260,NEGATIF
1,Bug setelah update. 1 . Turun pesawat ga kebag...,bug setelah update turun pesawat ga kebagian p...,bug setelah update turun pesawat tidak kebagia...,bug update turun pesawat kebagian parasut kara...,bug update turun pesawat bagi parasut karakter...,0.977045,NEGATIF
2,"Keseluruhan game nya udah bagus, apalagi setel...",keseluruhan game nya udah bagus apalagi setela...,keseluruhan game nya sudah bagus apalagi setel...,game bagus update grafik makinn mantap kekecew...,game bagus update grafik makinn mantap kecewa ...,0.886064,POSITIF
3,"Pubg setelah update bnyak aja masalahnya, sepe...",pubg setelah update bnyak aja masalahnya seper...,pubg setelah update banyak saja masalahnya sep...,pubg update bug map pas game player suka jalan...,pubg update bug map pas game player suka jalan...,0.988463,NEGATIF
4,Game bagus...tapi masih banyak bug...masih per...,game bagus tapi masih banyak bug masih perlu d...,game bagus tapi masih banyak bug masih perlu d...,game bagus bug diperbaiki mode tdm bug player ...,game bagus bug baik mode tdm bug player arena ...,0.673840,POSITIF
